### Task 1

1. Tải dữ liệu từ Hugging Face (cần downgrade `datasets` xuống version 3.2.0 để hỗ trợ remote code)

In [4]:
!pip install -q datasets==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from datasets import load_dataset

ds = load_dataset("conll2003", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

conll2003.py: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

2. Trích xuất câu và nhãn

In [6]:
train_sentences = ds['train']['tokens']
train_tags = ds['train']['ner_tags']
val_sentences = ds['validation']['tokens']
val_tags = ds['validation']['ner_tags']
test_sentences = ds['test']['tokens']
test_tags = ds['test']['ner_tags']

In [7]:
tag_names = ds['train'].features['ner_tags'].feature.names
idx2tag = {idx: tag for idx, tag in enumerate(tag_names)}
train_tag_names = []
for tag_ids in train_tags:
    train_tag_names.append([idx2tag[tag_id] for tag_id in tag_ids])
val_tag_names = []
for tag_ids in val_tags:
    val_tag_names.append([idx2tag[tag_id] for tag_id in tag_ids])
test_tag_names = []
for tag_ids in test_tags:
    test_tag_names.append([idx2tag[tag_id] for tag_id in tag_ids])

# sanity check
train_tag_names[:2]

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER']]

3. Xây dựng vocab

In [8]:
# word2idx
pad_token = '<PAD>'
pad_token_id = 0
unk_token = '<UNK>'
unk_token_id = 1

word2idx = {pad_token: pad_token_id, unk_token: unk_token_id}

words_set = set()
for sent in train_sentences:
    for word in sent:
        words_set.add(word)
word2idx.update({word: idx+2 for idx, word in enumerate(list(words_set))})

# tag2idx
tag_names = ds['train'].features['ner_tags'].feature.names
tag2idx = {tag: idx for idx, tag in enumerate(tag_names)}

print(f"Vocab size: {len(word2idx)}")
print(f"Num tag: {len(tag2idx)}")

Vocab size: 23625
Num tag: 9


### Task 2

1. Tạo lớp NER dataset

In [19]:
import torch
from torch.utils.data import Dataset

class NERDataset(Dataset):
    def __init__(self, sentences, tags, word2idx, tag2idx, pad_token='<PAD>', unk_token='<UNK>'):
        super().__init__()
        self.sentences = sentences
        self.tags = tags
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.pad_token = pad_token
        self.unk_token = unk_token

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tag = self.tags[idx]
        input_ids = torch.tensor([self.word2idx[token] if token in self.word2idx.keys() else self.word2idx[self.unk_token] for token in sentence], dtype=torch.long)
        tag_ids = torch.tensor([self.tag2idx[tag_id] for tag_id in tag], dtype=torch.long)
        return input_ids, tag_ids

# sanity check
dataset = NERDataset(train_sentences, train_tag_names, word2idx, tag2idx)
dataset[0]

(tensor([ 3043, 11874, 16177,  4916, 23167,  6324, 12205, 22790, 22699]),
 tensor([3, 0, 7, 0, 0, 0, 7, 0, 0]))

2. Tạo data loader

In [20]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn_with_padding(batch):
    sentences, tag_ids = zip(*batch)
    sentence_lengths = torch.tensor([len(sent) for sent in sentences], dtype=torch.long)

    # thực hiện padding
    # với nhãn, padding bằng giá trị -100, đây là giá trị mặc định được bỏ qua bởi cross entropy loss
    sentences_padded = pad_sequence(sentences, padding_value=word2idx[pad_token], batch_first=True)
    tag_ids_padded = pad_sequence(tag_ids, padding_value=-100, batch_first=True)

    return sentences_padded, tag_ids_padded, sentence_lengths

train_ds = NERDataset(train_sentences, train_tag_names, word2idx, tag2idx)
val_ds = NERDataset(val_sentences, val_tag_names, word2idx, tag2idx)
test_ds = NERDataset(test_sentences, test_tag_names, word2idx, tag2idx)

train_loader = DataLoader(train_ds, batch_size=8, collate_fn=collate_fn_with_padding, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=8, collate_fn=collate_fn_with_padding, shuffle=False, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=8, collate_fn=collate_fn_with_padding, shuffle=False, num_workers=2)

# sanity check
for input_ids, tag_ids, lengths in train_loader:
    print(input_ids.shape)
    print(tag_ids.shape)
    print(lengths)
    break

torch.Size([8, 33])
torch.Size([8, 33])
tensor([ 7,  8,  4, 26, 17, 33,  7,  3])


### Task 3

In [1]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class SimpleRNNForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes, dropout_p=0.3):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2idx[pad_token])
        self.rnn = nn.RNN(embedding_dim, hidden_size ,batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input_seqs, seq_lens):
        emb = self.dropout(self.embedding(input_seqs))
        packed_emb = pack_padded_sequence(emb, seq_lens, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.rnn(packed_emb) # (batch, seq len, hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        logits = self.fc(output) # fc tự động áp dụng lên dim cuối cùng
        return logits # (batch, seq len, num_classes)


Khởi tạo mô hình:

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

vocab_size = len(word2idx)
embedding_dim = 128
hidden_size = 256
num_classes = len(tag2idx)
model = SimpleRNNForTokenClassification(vocab_size, embedding_dim, hidden_size, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

print(model)

SimpleRNNForTokenClassification(
  (embedding): Embedding(23625, 128, padding_idx=0)
  (rnn): RNN(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=9, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


### Task 4 + 5

In [ ]:
import torch
from tqdm import tqdm

def train_one_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    pbar = tqdm(data_loader, desc='Training', total=len(data_loader))

    for input_ids, tag_ids, seq_lens in pbar:
        input_ids = input_ids.to(device)
        tag_ids = tag_ids.to(device)

        optimizer.zero_grad()

        logits = model(input_ids, seq_lens).view(-1, model.num_classes)
        targets = tag_ids.view(-1)
        loss = criterion(logits, targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    epoch_loss = total_loss / len(data_loader)
    return epoch_loss

@torch.no_grad()
def evaluate(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    pbar = tqdm(data_loader, desc='Evaluating', total=len(data_loader))
    total_correct = 0
    total_token = 0
    for input_ids, tag_ids, seq_lens in pbar:
        input_ids = input_ids.to(device)
        tag_ids = tag_ids.to(device)
        logits = model(input_ids, seq_lens).view(-1, model.num_classes)
        targets = tag_ids.view(-1)
        loss = criterion(logits, targets)
        total_loss += loss.item()
        # Tính accuracy
        preds = torch.argmax(logits, dim=-1)
        mask = targets != -100
        total_correct += (preds[mask] == targets[mask]).sum().item()
        total_token += mask.sum().item()

    validation_loss = total_loss / len(data_loader)
    accuracy = total_correct / total_token

    return validation_loss, accuracy

In [ ]:
epochs = 50
best_accuracy = 0.0
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}:")
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, accuracy = evaluate(model, val_loader, criterion, device)
    print(f"Train loss: {train_loss:.5f} | Val loss: {val_loss:.5f} | Accuracy: {accuracy:.5f}")

    # lưu lại mô hình tốt nhất dựa trên validation accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Best model saved with accuracy: {best_accuracy:.5f}")

    print("==="*20)

print(f"Training end. Best validation accuracy: {best_accuracy}")


Epoch 1/50:


Evaluating: 100%|██████████| 407/407 [00:01<00:00, 206.22it/s]


Train loss: 0.75726 | Val loss: 0.69846 | Accuracy: 0.84045
Best model saved with accuracy: 0.84045

Epoch 2/50:


Evaluating: 100%|██████████| 407/407 [00:01<00:00, 255.76it/s]


Train loss: 0.59142 | Val loss: 0.60948 | Accuracy: 0.84821
Best model saved with accuracy: 0.84821

Epoch 3/50:


...

Train loss: 0.09035 | Val loss: 0.28794 | Accuracy: 0.94107

Epoch 49/50:


Evaluating: 100%|██████████| 407/407 [00:01<00:00, 256.01it/s]


Train loss: 0.08825 | Val loss: 0.30253 | Accuracy: 0.94019

Epoch 50/50:


Evaluating: 100%|██████████| 407/407 [00:01<00:00, 218.08it/s]

Train loss: 0.08731 | Val loss: 0.27863 | Accuracy: 0.94229
Best model saved with accuracy: 0.94229
Training end. Best validation accuracy: 0.9422919668237218


In [ ]:
# lưu mô hình
torch.save(model.state_dict(), 'ner_part4.pth')

Test accuracy:

In [ ]:
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"\nTest loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

Evaluating: 100%|██████████| 432/432 [00:01<00:00, 261.78it/s]


Test loss: 0.4565457258746998
Test accuracy: 0.9204264024981157


Đánh giá bằng `seqeval`:

In [18]:
!pip install -q seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [41]:
from seqeval.metrics import classification_report

y_true = []
y_pred = []

for input_ids, tag_ids, seq_lens in test_loader:
    mask_ids = []  # mask để loại bỏ các vị trí padding
    for label in tag_ids:
        y_true.append([idx2tag[tag_id] for tag_id in label.tolist() if tag_id != -100])
        mask_ids.append(label != -100)

    input_ids = input_ids.to(device)
    tag_ids = tag_ids.to(device)

    logits = model(input_ids, seq_lens)
    preds = torch.argmax(logits, dim=-1)
    for pred, mask in zip(preds, mask_ids):
        y_pred.append([idx2tag[tag_id] for tag_id in pred[mask].cpu().tolist()])

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         LOC       0.03      0.03      0.03      1668
        MISC       0.02      0.01      0.01       702
         ORG       0.07      0.04      0.05      1661
         PER       0.01      0.02      0.01      1617

   micro avg       0.02      0.03      0.02      5648
   macro avg       0.03      0.03      0.03      5648
weighted avg       0.03      0.03      0.03      5648



Predict:

In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:
from nltk.tokenize import word_tokenize
import torch

@torch.no_grad()
def predict(model, tokenizer, sentence, word2idx, idx2word, idx2tag, device):
    model.eval()
    tokens = tokenizer(sentence)
    token_ids = [word2idx[token] if token in word2idx.keys() else word2idx['<UNK>'] for token in tokens]

    input_ids = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(device)
    seq_lens = torch.tensor([len(tokens)], dtype=torch.long)
    logits = model(input_ids, seq_lens)
    preds = torch.argmax(logits, dim=-1).squeeze(0)
    predicted_tags = [idx2tag[tag_id] for tag_id in preds.tolist()]
    return list(zip(tokens, predicted_tags))

sample_sentences = [
    "I love NLP",
    "This is Sparta!!!!!",
    "VNU University is located in Hanoi",
    "Paris is the capital city of France",
    "Linda is my pen pal",
    "The former director, Dr. Quill, flew from Kona International Airport in Hawaii to meet Senator P. O."
]

idx2word = {idx: word for word, idx in word2idx.items()}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

In [17]:
# load model từ checkpoint đã huấn luyện
import torch.optim as optim
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

vocab_size = len(word2idx)
embedding_dim = 128
hidden_size = 256
num_classes = len(tag2idx)
model = SimpleRNNForTokenClassification(vocab_size, embedding_dim, hidden_size, num_classes)
model.load_state_dict(torch.load('/content/ner_part4.pth'))
model = model.to(device)

for sent in sample_sentences:
    output = predict(model, word_tokenize, sent, word2idx, idx2word, idx2tag, device)
    print(output)

[('I', 'O'), ('love', 'B-ORG'), ('NLP', 'O')]
[('This', 'O'), ('is', 'I-PER'), ('Sparta', 'O'), ('!', 'O'), ('!', 'O'), ('!', 'O'), ('!', 'O'), ('!', 'O')]
[('VNU', 'B-PER'), ('University', 'I-ORG'), ('is', 'B-PER'), ('located', 'O'), ('in', 'O'), ('Hanoi', 'O')]
[('Paris', 'O'), ('is', 'I-PER'), ('the', 'I-PER'), ('capital', 'I-PER'), ('city', 'O'), ('of', 'O'), ('France', 'O')]
[('Linda', 'B-ORG'), ('is', 'I-PER'), ('my', 'O'), ('pen', 'O'), ('pal', 'O')]
[('The', 'O'), ('former', 'O'), ('director', 'O'), (',', 'O'), ('Dr.', 'O'), ('Quill', 'O'), (',', 'O'), ('flew', 'O'), ('from', 'O'), ('Kona', 'O'), ('International', 'O'), ('Airport', 'O'), ('in', 'O'), ('Hawaii', 'I-PER'), ('to', 'I-PER'), ('meet', 'I-PER'), ('Senator', 'O'), ('P.', 'O'), ('O', 'O'), ('.', 'I-PER')]


### Kết quả thực hiện (50 epochs):
- Độ chính xác trên tập validation: 0.9422 (94.22%)
- Độ chính xác trên tập test: 0.9204 (92.04%)

**Một điểm cần lưu ý:** Tuy accuracy trên tập validation và test rất cao nhưng đây không phải là một metrics phù hợp trong bài toán NER do đặc thù của bài toán là dữ liệu với nhãn O chiếm đa số, do đó, mô hình chỉ cần dự đoán đúng nhiều trên các nhãn này thì accuracy sẽ cao. Trong khi đó, các nhãn PER, ORG,... mới thực sự là các nhãn quan trọng.

Có thể thấy trong các ví dụ trên, mô hình dự đoán vẫn chưa được chính xác:
- Tên các địa danh như `Hanoi`, `Paris`, `France` bị gán nhãn O
- Tên tổ chức như `VNU` bị nhầm thành B-PER
- Tên người `Linda` bị nhầm thành tổ chức (B-ORG)  